# Exercise - Information Theory

<div class="alert alert-block alert-success">
This is an exercise in understanding the concept of entropy and discretication  of analog signals based on Chapter 5 “Information Theory”.

## Content <a id="sec_toc"> </a> 

[Entropy Calculation](#sec_a)

[Signal Discretization and Reconstruction](#sec_b)

[Entropy-based Signal Discretization](#sec_c)


In [ ]:
! pip install matplotlib

Import the needed libaries:

In [ ]:
from collections import Counter
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import convolve
from scipy.stats import entropy


## Entropy Calculation <a id="sec_a"></a> 

The entropy measures the uncertainty or unpredictability of the data. In a dataset, it represents \
the minimum number of bits required to encode or represent the data without losing information.


Formular of Entropy:  
$$ H(X) = - \sum_{i=1}^{n} p(x_i) \cdot log_2 (p(x_i))$$

$x_i$ ............. Specific / single outcome or symbol \
$p(x_i)$ ...... Probability of each outcome / symbol

A higher entropy value indicates more uncertainty, which implies more data is required to represent the system.

TASK: Choose a short sentence (string) and enter it where None is given.

In [ ]:
sentence = None

########################################### DO NOT CHANGE ########################################### 
# Method calculates the Entropy within a given sentence, either over given symbols or all symbols occuring
def get_entropy(string,symbols=None):
    if symbols is None:
        occur = dict(Counter(string.lower()))
        o = occur
    else: 
        occur = dict(Counter(char for char in string if char in symbols))
        o = {symbol: occur.get(symbol, 0) for symbol in symbols}
    H = 0
    print("Orruencies of Symbols: ", o)
    number_sym = sum(occur.values(),0)
    for f in occur.values():
        p = f/number_sym # len(string)
        H -= p*math.log2(p)
    return H

if sentence == None:
    print("Please enter a sentence!")
else:
    print("Your sentence: \"",sentence,"\"")
    print("Set of all occuring Symbols: ",set(sentence.lower()))
    print("Calculated Entropy: ",get_entropy(sentence))

The Entropy above is calculated automatically 

TASK: Define your own reduced set of symbols (at least six) within the given List. Enter the symbol within the ' '. \
Note: You can also add symbols that do not occure in your sentence.

In [ ]:
# List of reduced symbols
symbols = {'','','','','',''} # DEFINE the reduced symbol set (at least six different symbols)

########################################### DO NOT CHANGE ########################################### 
if sentence == None:
    print("Please enter a sentence in the cell before!")
else:
    print(f"Your sentence: \"{sentence}\"")
    print(f"Reduced set of Symbols: {symbols}")
    print(f"Reduced sentence: \"{''.join([char for char in sentence if char in symbols])}\" ")
    print(f"Calculated Entropy: {get_entropy(sentence,symbols)}")

[Back](#sec_toc)

## Signal Discretization and Reconstruction <a id="sec_b"></a> 

The upcoming section explores the process of converting a continuous analog sine wave into \
its discrete representation through discretization. This involves sampling the sine wave at \
specific intervals, effectively capturing its essence in a digital format. Furthermore, the \
discrete signal is leveraged to reconstruct the original analog waveform, showcasing the relationship \
between the sampled data and the continuous signal it represents. This process highlights the \
fundamental principles of signal processing and the interplay between analog and digital domains.

In [ ]:
# DO NOT CHANGE: class of pre defined analog signal 
class AnalogSignal:
    """
    A class to represent and manipulate an analog sine wave signal.
    
    Attributes:
    a: Amplitude of the sine wave in volts [V]. Default is 1.3.
    f: Frequency of the sine wave in hertz [Hz]. Default is 2.9.
    sF: Sampling frequency of the signal in hertz [Hz]. Default is 1000.
    d: Duration of the signal in seconds [s]. Default is 2.
    
    Methods:
    get_signal: Computes the sine wave values for given timestamps or generates timestamps if not provided.
    get_frequency: Returns the frequency of the signal.
    get_timestamps: Generates an array of timestamps based on duration and sampling frequency.
    sample_signal: Samples the analog signal at a specified sampling frequency and returns the sampled values and timestamps.
    """

    def __init__(self, duration=2, sampling_frequency=1000):
        self.a = 1.3 # amplitude of signal [V]
        self.f = 3.0 # frequency of signal [Hz]
        self.sF = sampling_frequency # Sampling rate of signal [Hz]
        self.d = duration # Duration of signal [s]
    
    def get_signal(self, t = None, phase=0):
        if t is None: 
            t=self.get_timestamps(phase)
        signal = self.a * np.sin(2 * np.pi * self.f * t)
        return signal
    
    def get_frequency(self):
        return self.f
    
    def get_timestamps(self, phase=0):
        return np.linspace(0+phase, self.d, int(self.d*self.sF), endpoint=False)

    def sample_signal(self, sample_frequency):
        phase = 0
        f = self.sF
        self.sF = sample_frequency
        signal = self.get_signal(phase=phase)
        t = self.get_timestamps(phase)
        self.sF = f
        return signal, t

def create_plot(title, duration = 1, ylim = 1):
    """
    Creates/ Configures a plot (template) for visualizing a signal. (does not display it)

    Parameters:
    title: The title of the plot.
    duration: The duration of the signal in seconds [s] to be displayed on the x-axis.

    Returns: None
    """
    plt.figure(figsize=(10, 6))
    plt.title(title)
    plt.xlabel('Time [s]')
    plt.ylabel('Amplitude [V]')
    plt.grid(alpha=0.3)
    plt.ylim(-ylim, ylim)
    plt.xlim(0, math.floor(duration))
    plt.axhline(0, color='black', lw=0.5, ls='-')

Below is a Vizualization of the given analog signal.

In [ ]:
# DO NOT CHANGE: Creating analog sine signal
duration = 2.0 # Duration of signal [s]

# Get signal
analogSignal = AnalogSignal(duration = duration)
# Plot signal
create_plot('Analog Signal - Sine Wave', duration, 1.5)
plt.plot(analogSignal.get_timestamps(), analogSignal.get_signal(), color="orange")
plt.show()

The Nyquist-Shannon Theorem states that the sampling rate must be at least twice the \
highest frequency present in the signal to ensure accurate discretization. Based on this \
principle, we calculate the signal's frequency and sample it at the specified rate. The \
resulting plot visually compares the original analog signal with its sampled counterpart.

TASK: Explore different sample frequencies by changing the frequency multiplyer.

Note: &ensp; $f_S = multiply \cdot f_A$  &emsp; ($f_S$....sample frequency, $f_A$....analog frequency)

In [ ]:
multiply = None # CHANGE the multiplicator of frequency to a reasonable value


############################### DO NOT CHANGE ##############################
if (multiply==None):
    print("The multiply is still None. Please enter a reasonable value!")
else:
    max_frequency = analogSignal.get_frequency() # Getting frequency of signal

    sample_frequency = max_frequency * multiply # Calculating sample frequency

    discrete_signal_values, t_discrete = analogSignal.sample_signal(sample_frequency) # discritice signal with sample_frequency

    # Print given frequencies
    print(f"Frequency of Signal: {max_frequency:.4} Hz")
    print(f"Sampling Frequency: {sample_frequency:.4}  Hz")

    # Plot Signals
    create_plot('Analog vs. Discretized Signal', duration, 1.5)
    plt.plot(analogSignal.get_timestamps(), analogSignal.get_signal(), label="Analog Signal", color="orange")
    for x in t_discrete:
        plt.axvline(x=x,color='green', linestyle='--', linewidth=0.8)
    plt.scatter(t_discrete,discrete_signal_values, label="Discrete sampled values", color="green")
    plt.legend(loc='upper right')
    plt.show()

As demonstrated in the following code snipped, the signal is reconstructed using the sinc function. \
Additionally, the mean squared error (MSE) is calculated to evaluate the difference between the \
original signal and its reconstructed counterpart.

In [ ]:
# DO NOT CHANGE: Reconstruction of Signal
T = 1/sample_frequency
n = np.arange(len(discrete_signal_values)) 
t = np.linspace(0,duration, 1000)
recon_signal_values = np.zeros_like(t)
for i in range(len(t)):
   recon_signal_values[i] = np.sum(discrete_signal_values * np.sinc((t[i] - n * T) / T))

analog_signal_values = analogSignal.get_signal(t=t)
mse = np.mean((analog_signal_values - recon_signal_values) ** 2)
print(f"Mean Squared Error (MSE) of original vs. reconstructed signal: {mse:.4f} V²")

# Plot Signals
create_plot('Reconstructed Signal', duration, 1.5)
plt.plot(t,recon_signal_values ,label="Reconstructed")
plt.plot(analogSignal.get_timestamps(), analogSignal.get_signal(), ls=':', color='orange',label="Original" )
plt.legend(loc='upper right')
plt.show()

[Back](#sec_toc)

## Entropy-based Signal Discretization

Entropy and signal discretization are closely related concepts that can be effectively combined to\
improve signal processing. By applying entropy-based discretization, we can convert a continuous \
signal into a discrete one while preserving the most significant features and minimizing information \
loss. This method involves dividing the signal into a set of bins and assigning each value to the most \
appropriate bin, with the bin centers used for reconstruction. By linking entropy with discretization, \
we can optimize the representation of the signal, ensuring that the discretized version retains as much \
of the original information as possible, making it ideal for tasks like signal compression, reconstruction, \
and analysis.

In [ ]:
# DO NOT CHANGE: Overlapped analog signal
class SignalProcessor:
    def __init__(self, duration, sampling_rate):
        self.duration = duration
        self.sampling_rate = sampling_rate
        self.time = np.linspace(0, duration, int(duration * sampling_rate), endpoint=False)

    def sinusoidal(self, amplitude, frequency, delay):
        """Generates a sinusoidal signal."""
        return amplitude * np.sin(2 * np.pi * frequency * (self.time - delay))

    def rectangular_pulse_train(self, peak, impulse_length, impulse_interval):
        """Generates a rectangular pulse train signal."""
        signal = np.zeros_like(self.time)
        for start in np.arange(0, self.duration, impulse_interval):
            end = start + impulse_length
            signal[(self.time >= start) & (self.time < end)] = peak
        return signal

    def triangular_pulse_train(self, peak, impulse_length, impulse_interval):
        """Generates a triangular pulse train signal."""
        signal = np.zeros_like(self.time)
        for start in np.arange(0, self.duration, impulse_interval):
            end = start + impulse_length
            mid = (start + end) / 2
            for t in self.time[(self.time >= start) & (self.time < end)]:
                if t <= mid:
                    signal[np.isclose(self.time, t)] = peak * (t - start) / (mid - start)
                else:
                    signal[np.isclose(self.time, t)] = peak * (end - t) / (end - mid)
        return signal

    def sawtooth_pulse_train(self, peak, impulse_length, impulse_interval):
        """Generates a sawtooth pulse train signal."""
        signal = np.zeros_like(self.time)
        for start in np.arange(0, self.duration, impulse_interval):
            end = start + impulse_length
            for t in self.time[(self.time >= start) & (self.time < end)]:
                signal[np.isclose(self.time, t)] = peak * (t - start) / (end - start)
        return signal

    def half_sinusoidal_pulse_train(self, amplitude, frequency, impulse_interval):
        """Generates a half-sinusoidal pulse train signal."""
        signal = np.zeros_like(self.time)
        for start in np.arange(0, self.duration, impulse_interval):
            half_sinus = amplitude * np.sin(2 * np.pi * frequency * (self.time - start))
            half_sinus = np.clip(half_sinus, 0, None)  # Keep only the positive half
            signal += np.where((self.time >= start) & (self.time < start + 1/frequency), half_sinus, 0)
        return signal

    def compute_convolution(self, signal1, signal2):
        """Computes the convolution of two signals and normalizes the result."""
        conv = convolve(signal1, signal2, mode='full')
        conv = conv[len(conv)//2 - len(self.time)//2 : len(conv)//2 + len(self.time)//2]  # Align time axis
        conv /= np.max(np.abs(conv))  # Normalize the convolution result
        return conv

    def compute_elementwise_product(self, signal1, signal2):
        """Computes the element-wise product of two signals."""
        return signal1 * signal2

    def entropy_based_discretization(self, signal, num_bins):
        """Performs entropy-based discretization of a signal."""
        # Define bin edges for discretization
        bin_edges = np.linspace(np.min(signal), np.max(signal), num_bins + 1)
        
        # Digitize signal into bins
        digitized = np.digitize(signal, bin_edges) - 1
        digitized = np.clip(digitized, 0, num_bins - 1)  # Ensure indices are within valid range
        # Reconstruct signal using bin centers
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
        reconstructed_signal = bin_centers[digitized]
        # Compute probabilities for each bin
        bin_counts = np.bincount(digitized, minlength=num_bins)
        probabilities = bin_counts / len(signal)
        # Compute entropy for the signal
        signal_entropy = entropy(probabilities, base=2)
        num_steps=self.count_signal_steps(reconstructed_signal)
        return reconstructed_signal, signal_entropy, bin_edges, bin_counts, num_steps

    def uniform_discretization(self, signal, step_size):
        """Performs classic uniform discretization of a signal."""
        indices = np.arange(0, len(signal), step_size)
        sampled_values = signal[indices]
        reconstructed_signal = np.zeros_like(signal)
        for i in range(len(indices) - 1):
            reconstructed_signal[indices[i]:indices[i+1]] = sampled_values[i]
        reconstructed_signal[indices[-1]:] = sampled_values[-1]  # Fill the last section
        return reconstructed_signal
    
    def get_time(self):
        return self.time

    def count_signal_steps(self,reconstructed_signal):
        """Zählt die Stufen im rekonstruierten Signal."""
        # Identifiziere Übergänge: wo sich der Wert im Signal ändert
        step_changes = np.diff(reconstructed_signal) != 0
        # Zähle die Anzahl der Änderungen und addiere die erste Stufe
        num_steps = np.sum(step_changes) + 1  # +1 für die erste Stufe
        return num_steps

def plot_entropy_discretization(time, signal, reconstructed_signal, bin_edges, bin_counts):
    """
    Plots the original signal with the bins and their counts as a bar plot, 
    with interactivity to toggle visibility of individual signals.
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), gridspec_kw={"width_ratios": [3, 1]})

    # Plot the original and reconstructed signals
    ax1.axhline(0, color='black', lw=0.5, ls='-')
    line1, = ax1.plot(time, signal, label="Original Signal", alpha=0.7, color='orange')
    line2, = ax1.plot(time, reconstructed_signal, label="Reconstructed Signal", color='C0', linestyle="--")
    ax1.set_xlabel("Time [s]")
    ax1.set_ylabel("Amplitude [V]")
    ax1.legend(loc="upper right")

    # Add bar plot of bin counts
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    ax2.barh(bin_centers, bin_counts, height=np.diff(bin_edges), color="green", alpha=0.7, edgecolor="black")
    ax2.set_xlabel("Bin Counts")
    ax2.set_ylabel("Amplitude Range")
    ax2.set_title("Bin Count Distribution")
    ax2.grid(alpha=0.3)

    plt.title("Entropy-based Discretization with Bin Counts")
    plt.grid(alpha=0.3)
    plt.show()
    
def calculate_mse(original_signal, reconstructed_signal):
        """Calculates the Mean Squared Error between two signals."""
        return np.mean((original_signal - reconstructed_signal) ** 2)

def plot_signals(time, *signals, labels, colors, linestyles, title, duration):
    """Plots multiple signals."""
    create_plot(title, duration, 1.2)
    for signal, label, color, ls in zip(signals, labels, colors, linestyles):
        plt.plot(time, signal, label=label, color = color, linestyle=ls)
    plt.legend(loc="upper right")
    plt.show()

The next code section generates a combined signal by superimposing multiple signal types, such \
as sinusoidal waves, rectangular pulses, or triangular pulses, to create a complex waveform for analysis.

In [ ]:
# DO NOT CHANGE: Generating combined signal

dur = 200  # [s]
sampling_rate = 200  # [Hz]
sp = SignalProcessor(dur, sampling_rate)
# Create seperate part-signals
sinus = sp.sinusoidal(amplitude=1, frequency=0.2, delay=0.5)
rect_pulse = sp.rectangular_pulse_train(peak=1, impulse_length=50, impulse_interval=100)
tri_pulse = sp.triangular_pulse_train(peak=1, impulse_length=50, impulse_interval=100)
saw_pulse = sp.sawtooth_pulse_train(peak=1, impulse_length=50, impulse_interval=100)
sine_pulse = sp.half_sinusoidal_pulse_train(amplitude=1, frequency=0.01, impulse_interval=100)
# Combine signals
elementwise_product = sp.compute_elementwise_product(sinus, sine_pulse)

# Plot original signal
create_plot("Non-uniform / superimposed analog signal", dur, 1.3)
plt.plot(sp.get_time(), elementwise_product, color="orange")
plt.show()

The following code section performs entropy-based discretization and reconstruction of a signal. It \
divides the signal into bins based on entropy optimization, assigns discrete values, and reconstructs \
the signal using the bin centers.

TASK: Change the number of bins used for the discritization based on the entropy.

In [ ]:
bin_number = None # CHANGE the bin number in a range between (2,20)

# DO NOT CHANGE
if (bin_number == None):
    print("The bin_number is still None. Please enter a valid value!")
elif(bin_number<=1 or bin_number>20):
    print("The bin_number is out of the given range. Please enter a valid value!")
else: 
    # Perform entropy-based discretization
    reconstructed_entropy_signal, signal_entropy, bin_edges, bin_counts, num_entropy_steps = sp.entropy_based_discretization(elementwise_product, num_bins=bin_number)
    print(f"Signal Entropy: {signal_entropy}")

    # Plot the entropy-based discretization
    plot_entropy_discretization(sp.get_time(),elementwise_product, reconstructed_entropy_signal, bin_edges, bin_counts)

The next code section compares uniform discretization with entropy-based discretization, ensuring that both reconstructed signals have the same number of "steps" (distinct levels). This allows for a fair evaluation of the methods' performance and reconstruction accuracy.

In [ ]:
# DO NOT CHANGE: Compare reconstructions

step_size = round((num_entropy_steps/19902)**(1/-1)) # Power function as a relationship between num_uniform_steps and step_size

# Perform uniform discretization
reconstructed_uniform_signal = sp.uniform_discretization(elementwise_product, step_size=step_size)
# Plot the original and reconstructed signals
plot_signals(sp.get_time(),elementwise_product, reconstructed_entropy_signal, reconstructed_uniform_signal, labels=['Original Signal', 'Entropy-based Reconstruction', 'Uniform Reconstruction'], colors=["orange","C0","brown"], linestyles=["-","-","-"], title = "Uniform vs. Entropy-based discritization", duration = dur)
# Plot the original and reconstructed signals
plot_signals(sp.get_time(),elementwise_product, reconstructed_entropy_signal, reconstructed_uniform_signal, labels=['Original Signal', 'Entropy-based Reconstruction', 'Uniform Reconstruction'], colors=["orange","C0","brown"], linestyles=["-","-","-"], title = "Uniform vs. Entropy-based discritization (Zoomed)", duration = dur/2-30)

# Calculate MSE
mse_entropy = calculate_mse(elementwise_product, reconstructed_entropy_signal)
mse_uniform = calculate_mse(elementwise_product, reconstructed_uniform_signal)
print(f"MSE (Entropy-based): {mse_entropy} V²")
print(f"MSE (Uniform): {mse_uniform} V²")

[Back](#sec_toc)